<div><img src = "https://jacademy.juventus.com/media/images/juventus-academy-toronto/Home%20page%20banner.jpg" 
        width = 500 alt="Toronto Skyline"> </div>
<h1 align=center><font size = 5>Segmenting & Clustering Neighborhoods in Toronto</font></h1>

This notebook is the 1st part of the ***Capstone Project in Applied Data Science for Coursera & IBM***
The aim is to obtain information about the neighborhoods in **Toronto, Canada**, so i can use this information to further analyse the venues available.
## 3. Explore final Dataframe with coordinates of postal codes from Toronto
After creating the final Dataframe, I will explore the data #to do

Firstly and foremost I need the following libraries:

In [1]:
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
import requests
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Now I load the csv I created with the final Dataframe:

In [2]:
#read the csv
df_final = pd.read_csv('PC_N_Toronto_coord.csv').iloc[:,1:]
df_final.head()

,PostalCode,Borough,Neigborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [3]:
# Now I check if there are any null values
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 5 columns):
PostalCode     103 non-null object
Borough        103 non-null object
Neigborhood    103 non-null object
Latitude       103 non-null float64
Longitude      103 non-null float64
dtypes: float64(2), object(3)
memory usage: 4.1+ KB


I'll use geopy library to get the latitude and longitude values of Toronto City.

In order to define an instance of the geocoder, I need to define a user_agent. I will name my agent <em>tor_explorer</em>, as shown below.

In [4]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


I'll Create a map of Toronto with its neighborhoods on top as POI.

In [5]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighd in zip(df_final['Latitude'], df_final['Longitude'], 
                                       df_final['Borough'], df_final['Neigborhood']):
    label = '{}, {}'.format(neighd, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng], radius=5, 
                        popup=label, 
                        color='blue', 
                        fill=True, 
                        fill_color='#3186cc', 
                        fill_opacity=0.5, 
                        parse_html=False).add_to(map_toronto)  
    
map_toronto

First I'll define the required Foursquare Credentials and Version to use its request url

In [6]:
CLIENT_ID = '4QFEINHYB35HFDKHZQYHUSKX5UWGPTCFUCMXSIRHS5CQJJ51'
CLIENT_SECRET = 'XUIM0KSEAAPNBMC0XXDK3QMS3XDYEHGRUG1VABVMKJMVTSCI'
VERSION = '20190618'
LIMIT = 100 # number of maximum venues to be returned for every neighborhood
print('Credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Credentials:
CLIENT_ID: 4QFEINHYB35HFDKHZQYHUSKX5UWGPTCFUCMXSIRHS5CQJJ51
CLIENT_SECRET:XUIM0KSEAAPNBMC0XXDK3QMS3XDYEHGRUG1VABVMKJMVTSCI


### 3.1. Exploring Neighborhoods in Toronto

I'll create a function to repeat the process of pulling all venues for all the neighborhoods in Toronto

In [7]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        # creation of the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now I'll retrieve, with the above function, every venue for each neighborhood and 
create a new dataframe with this information called *toronto_venues*.

In [8]:
toronto_venues = getNearbyVenues(names = df_final['Neigborhood'],
                                   latitudes = df_final['Latitude'],
                                   longitudes = df_final['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

The resulting Dataframe was created and now I'll check its shape and look at the first records

In [9]:
print(toronto_venues.shape)
toronto_venues.head()

(2258, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge, Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
3,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place


But I want to know also the venues that were returned by Neighborhood and by descending order

In [10]:
toronto_venues.groupby('Neighborhood')\
    .count().sort_values(by=['Venue'],axis=0, ascending=False).head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
St. James Town,100,100,100,100,100,100
"Ryerson, Garden District",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100


In [11]:
toronto_venues.groupby('Neighborhood')\
    .count().sort_values(by=['Venue'],axis=0, ascending=False).tail()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Humber Bay, King's Mill Park, Kingsway Park South East, Mimico NE, Old Mill South, The Queensway East, Royal York South East, Sunnylea",1,1,1,1,1,1
"Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park",1,1,1,1,1,1
"Silver Hills, York Mills",1,1,1,1,1,1
Scarborough Village,1,1,1,1,1,1
"Emery, Humberlea",1,1,1,1,1,1


Looks like i have to filter out the neighborhoods that don't have enough venues... 
let's say... 7 as minimum.

In [12]:
filter_out_df = toronto_venues.groupby('Neighborhood')\
    .count()[['Venue']]
filter_out_list = list(filter_out_df[filter_out_df['Venue'] < 7].index)

In [13]:
toronto_venues = toronto_venues[~toronto_venues['Neighborhood'].isin(filter_out_list)].copy()

Now I want to find out how many unique categories I retrieved

In [14]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 261 uniques categories.


### 3.2 Analysis by Neighborhood

In [15]:
# dummies encoding
toronto_de = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
# add the neighborhood column back to dataframe
toronto_de['Neighborhood'] = toronto_venues['Neighborhood'] 
# place columns in the right order
order = [toronto_de.columns[-1]] + list(toronto_de.columns[:-1])
toronto_de = toronto_de[order]
toronto_de.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2099 entries, 4 to 2254
Columns: 261 entries, Yoga Studio to Women's Store
dtypes: object(1), uint8(260)
memory usage: 565.7+ KB


And let's examine the new dataframe size.

In [16]:
toronto_de.shape

(2099, 261)

Grouping rows by neighborhood and mean of the frequency of occurrence of each category, I'll get
a better picture of what I retrieved

In [17]:
toronto_grouped = toronto_de.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.04,0.0,0.0,0.01,0.01,0.0,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.03,0.000000,0.04,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.01,0.0,0.01,0.02,0.0,0.000000,0.0,0.03,0.01,0.

And the following is the size of the grouping Dataframe

In [18]:
toronto_grouped.shape

(55, 261)

To better evaluate the neighborhoods I'll want to obtain a dataframe with the top 7 venues for each

In [19]:
num_top_venues = 7

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
toronto_venues_sorted = pd.DataFrame(columns=columns)
toronto_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    toronto_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

toronto_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Steakhouse,American Restaurant,Thai Restaurant,Hotel
1,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Fried Chicken Joint,Coffee Shop,Pharmacy,Pizza Place,Sandwich Place,Liquor Store
2,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Sandwich Place,Pub,Pool,Skating Rink,Dance Studio
3,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Frozen Yogurt Shop,Diner,Sandwich Place,Fast Food Restaurant,Bridal Shop,Bank
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Fast Food Restaurant,Thai Restaurant,Butcher,Pub,Café


In [20]:
toronto_venues_sorted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 8 columns):
Neighborhood             55 non-null object
1st Most Common Venue    55 non-null object
2nd Most Common Venue    55 non-null object
3rd Most Common Venue    55 non-null object
4th Most Common Venue    55 non-null object
5th Most Common Venue    55 non-null object
6th Most Common Venue    55 non-null object
7th Most Common Venue    55 non-null object
dtypes: object(8)
memory usage: 3.5+ KB


## 4. Clustering Neighborhoods with k-means

I'll run *k*-means to cluster the neighborhood into 5 clusters.

In [21]:
# set number of clusters
kclusters = 3
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
# seeing cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 0, 2, 0, 2, 2, 0, 2, 2])

Now I create a dataframe that contains the cluster and the top 7 venues for each neighborhood.

In [22]:
# add clustering labels
toronto_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = df_final[~df_final['Neigborhood'].isin(filter_out_list)].copy()
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(toronto_venues_sorted.set_index('Neighborhood'), on='Neigborhood', how='inner')
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].fillna(0).astype('int')
toronto_merged.head()

,PostalCode,Borough,Neigborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1,Electronics Store,Rental Car Location,Intersection,Mexican Restaurant,Tech Startup,Medical Center,Breakfast Spot
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,2,Athletics & Sports,Bank,Lounge,Hakka Restaurant,Thai Restaurant,Bakery,Caribbean Restaurant
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,2,Bakery,Bus Line,Park,Intersection,Fast Food Restaurant,Metro Station,Bus Station
11,M1R,Scarborough,"Maryvale, Wexford",43.750071,-79.295849,1,Middle Eastern Restaurant,Auto Garage,Sandwich Place,Shopping Mall,Breakfast Spot,Bakery,Dog Run
13,M1T,Scarborough,"Clarks Corners, Sullivan, Tam O'Shanter",43.781638,-79.304302,0,Pizza Place,Fried Chicken Joint,Noodle House,Bank,Thai Restaurant,Chinese Restaurant,Fast Food Restaurant


Finally, I'll plot the results

In [23]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], 
                                  toronto_merged['Neigborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.5).add_to(map_clusters)
       
map_clusters

### 4.1 Analyse Clusters Obtained

Now, I have 3 clusters which I'll provide a summarizing Name. 

#### Cluster #1

In [24]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, 
                   toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neigborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
13,"Clarks Corners, Sullivan, Tam O'Shanter",0,Pizza Place,Fried Chicken Joint,Noodle House,Bank,Thai Restaurant,Chinese Restaurant,Fast Food Restaurant
15,L'Amoreaux West,0,Fast Food Restaurant,Chinese Restaurant,Thrift / Vintage Store,Grocery Store,Coffee Shop,Pharmacy,Pizza Place
35,"Woodbine Gardens, Parkview Hill",0,Fast Food Restaurant,Pizza Place,Gastropub,Gym / Fitness Center,Pharmacy,Café,Breakfast Spot
42,"The Beaches West, India Bazaar",0,Park,Pub,Ice Cream Shop,Brewery,Sandwich Place,Fast Food Restaurant,Burger Joint
47,Davisville,0,Pizza Place,Dessert Shop,Sandwich Place,Restaurant,Café,Thai Restaurant,Italian Restaurant
62,"Bedford Park, Lawrence Manor East",0,Italian Restaurant,Coffee Shop,Fast Food Restaurant,Thai Restaurant,Butcher,Pub,Café
87,Business Reply Mail Processing Centre 969 Eastern,0,Yoga Studio,Butcher,Smoke Shop,Light Rail Station,Brewery,Spa,Farmers Market
89,"Alderwood, Long Branch",0,Pizza Place,Coffee Shop,Sandwich Place,Pub,Pool,Skating Rink,Dance Studio
101,"Albion Gardens, Beaumond Heights, Humbergate, ...",0,Grocery Store,Fried Chicken Joint,Coffee Shop,Pharmacy,Pizza Place,Sandwich Place,Liquor Store


This cluster seems to be a residential one, suburbs maybe...

#### Cluster #2

In [25]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, 
                   toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neigborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
2,"Guildwood, Morningside, West Hill",1,Electronics Store,Rental Car Location,Intersection,Mexican Restaurant,Tech Startup,Medical Center,Breakfast Spot
11,"Maryvale, Wexford",1,Middle Eastern Restaurant,Auto Garage,Sandwich Place,Shopping Mall,Breakfast Spot,Bakery,Dog Run
27,"Flemingdon Park, Don Mills South",1,Coffee Shop,Gym,Beer Store,Asian Restaurant,Grocery Store,Sandwich Place,Japanese Restaurant
36,Woodbine Heights,1,Pharmacy,Athletics & Sports,Cosmetics Shop,Curling Ice,Bus Stop,Skating Rink,Beer Store
38,Leaside,1,Sporting Goods Shop,Coffee Shop,Grocery Store,Sushi Restaurant,Furniture / Home Store,Burger Joint,Mexican Restaurant
39,Thorncliffe Park,1,Indian Restaurant,Yoga Studio,Supermarket,Grocery Store,Gym,Housing Development,Intersection
45,Davisville North,1,Gym,Hotel,Clothing Store,Playground,Sandwich Place,Breakfast Spot,Food & Drink Shop
46,North Toronto West,1,Coffee Shop,Yoga Studio,Bagel Shop,Fast Food Restaurant,Diner,Dessert Shop,Mexican Restaurant
71,"Lawrence Heights, Lawrence Manor",1,Clothing Store,Furniture / Home Store,Women's Store,Event Space,Accessories Store,Vietnamese Restaurant,Fraternity House
92,"Kingsway Park South West, Mimico NW, The Queen...",1,Wings Joint,Supplement Shop,Convenience Store,Burrito Place,Burger Joint,Sandwich Place,Discount Store


This cluster looks like a mash up of High end residential, with business area, and tourism. Looks like an high income area

#### Cluster #3

In [26]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, 
                   toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neigborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
4,Cedarbrae,2,Athletics & Sports,Bank,Lounge,Hakka Restaurant,Thai Restaurant,Bakery,Caribbean Restaurant
7,"Clairlea, Golden Mile, Oakridge",2,Bakery,Bus Line,Park,Intersection,Fast Food Restaurant,Metro Station,Bus Station
18,"Fairview, Henry Farm, Oriole",2,Clothing Store,Fast Food Restaurant,Coffee Shop,Restaurant,Bakery,Japanese Restaurant,Kids Store
22,Willowdale South,2,Sushi Restaurant,Coffee Shop,Restaurant,Ramen Restaurant,Sandwich Place,Café,Japanese Restaurant
28,"Bathurst Manor, Downsview North, Wilson Heights",2,Coffee Shop,Frozen Yogurt Shop,Diner,Sandwich Place,Fast Food Restaurant,Bridal Shop,Bank
29,"Northwood Park, York University",2,Bar,Massage Studio,Miscellaneous Shop,Metro Station,Caribbean Restaurant,Falafel Restaurant,Coffee Shop
41,"The Danforth West, Riverdale",2,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Dessert Shop,Bubble Tea Shop
43,Studio District,2,Café,Coffee Shop,Gastropub,American Restaurant,Bakery,Italian Restaurant,Cheese Shop
49,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",2,Pub,Coffee Shop,American Restaurant,Supermarket,Light Rail Station,Fried Chicken Joint,Bagel Shop
51,"Cabbagetown, St. James Town",2,Coffee Shop,Restaurant,Italian Restaurant,Bakery,Pub,Café,Pizza Place


This cluster looks like a mash up of comercial area, and tourism. Looks like a city center or a major hub.